The Widgets module has the graphing tools, matplotlib has the backend for graphs

In [1]:
import matplotlib.pyplot as plt

import spikeinterface.full as si
import spikeinterface.extractors as se
import spikeinterface.widgets as sw
from spikeinterface.sortingcomponents.peak_detection import detect_peaks


In [2]:
sw.set_default_plotter_backend(backend="matplotlib")

Get data and make sorting. Make sure to add your path to the data

In [ ]:
FILE_PATH_TO_DATA = "file path here"

recording = se.BlackrockRecordingExtractor('FILE_PATH_TO_DATA')

# Convert to NumpyRecording to ensure proper shape
traces = recording.get_traces()
recording_2d = NumpyRecording(traces_list=traces.reshape(-1, 1), sampling_frequency=recording.get_sampling_frequency())

# Get a chunk of data to calculate noise level
chunk_traces = recording_2d.get_traces(start_frame=0, end_frame=30000)  # 1 second of data
noise_level = np.median(np.abs(chunk_traces - np.median(chunk_traces))) / 0.6744897501960817  # MAD estimator

# Detect peaks with manual noise level using 'by_channel' method with lower threshold
peaks = detect_peaks(recording_2d, method='by_channel', noise_levels=np.array([noise_level]), 
                    detect_threshold=1.5, peak_sign='neg',  # Lower threshold and focus on negative peaks
                    exclude_sweep_ms=0.5)  # Increase refractory period

# Create sorting using NumpySorting with channel IDs as unit IDs
sorting = NumpySorting.from_peaks(peaks, recording_2d.get_sampling_frequency(), unit_ids=recording_2d.channel_ids)


make a sorting analyzer

In [ ]:
analyzer = si.create_sorting_analyzer(sorting=sorting, recording=recording, format="memory")
# core extensions
analyzer.compute(["random_spikes", "waveforms", "templates", "noise_levels"])

# more extensions
analyzer.compute(["spike_amplitudes", "unit_locations", "spike_locations", "template_metrics"])

plot traces

In [ ]:
w_traces = sw.plot_traces(recording)

In [ ]:
raster = sw.plot_rasters(sorting)

In [ ]:
w_el = sw.plot_probe_map(recording)


In [ ]:
sw.plot_unit_waveforms(analyzer, unit_ids=sorting.unit_ids[:5], figsize=(16, 4))

We can filter the data with a bandpass

In [ ]:
rec_filtred = si.bandpass_filter(recording=recording, freq_min=300.0, freq_max=6000.0, margin_ms=5.0)

peaks = detect_peaks(
    recording=rec_filtred,
    method="locally_exclusive",
    peak_sign="neg",
    detect_threshold=6,
    exclude_sweep_ms=0.3,
    radius_um=100,
    noise_levels=None,
    random_chunk_kwargs={},
    chunk_memory="10M",
    n_jobs=1,
    progress_bar=True,
)

In [ ]:
si.plot_peak_activity(recording=rec_filtred, peaks=peaks)

plt.show()


In [ ]:
anim = si.plot_peak_activity(recording=rec_filtred, peaks=peaks, bin_duration_s=5)


plt.show()